Nama  : San Antonio Limbong

NIM   : 12S19033

### Exercise 1 | Parsing
#### Exercise 1.1 | Ubiquitous Ambiguity

In [1]:
import nltk
groucho_grammar = nltk.CFG.fromstring("""
S -> NP NP
PP -> N NP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
""")

Kode program diatas menunjukkan  bagaimana ambiguitas dalam frasa:  I shot an elephant in my pajamas. Yang pertama diperlukan adalah mendefinsikan tata bahasa sederhana atau simple grammar.

Adapun Contoh ambiguity yang terkenal ditunjukkan  dari film Groucho Marx, Animal Crackers (1930) 

While hunting in Africa, I shot an elephant in my pajamas. How he got into my pajamas, I don't know.

In [2]:
from sklearn import tree

sent = ['I', 'shot', 'an', 'elephant', 'in', 'pajamas']
parser = nltk.ChartParser(groucho_grammar)
trees = parser.parse(sent)
for t in trees:
    print(t)
    t.draw()

Kalimat diatas memiliki Grammar yang mmeungkinkan kita untuk menganalisis dalam dua cara yaitu tergantung pada apakah frasa preposisi  in my pajamas menggambarkan gajah atau peristiwa penembakan.

#### Exercise 1.2 | Recursive Descent Parsing with Context Free Grammar

In [3]:
grammar1 = nltk.CFG.fromstring("""
S -> NP NP
VP -> V NP | V NP PP
PP -> P NP
V -> "saw" | "ate" | "walked"
NP -> "John" | "Mary" | "Bob" | Det N | Det N PP
Det -> "a" | "an" | "the" | "my"
N -> "man" | "dog" | "cat" | "telescope" | "park"
P -> "in" | "on" | "by" | "with"
""")

sent = "Mary saw Bob".split()
rd_parser = nltk.RecursiveDescentParser(grammar1)
for t in rd_parser.parse(sent):
    print(t)
    t.draw()

Sebuah parser memproses kalimat masukan sesuai dengan produksi grammar, dan membangun satu atau lebih struktur konstituen yang sesuai dengan grammar. Sebuah parser memungkinkan grammar untuk dievaluasi terhadap kumpulan test sentences, membantu linguist untuk menemukan kesalahan dalam analisis grammar.

Jenis pengurai yang paling sederhana menafsirkan grammar sebagai spesifikasi tentang cara memecah tujuan tingkat tinggi (high-level) menjadi beberapa subtujuan yang paling rendah (lower-level). Tujuan dari top-level adalah untuk menemukan s. 

In [4]:
sent = "the dog saw a man in the park".split()
rd_parser = nltk.RecursiveDescentParser(grammar1)
for t in rd_parser.parse(sent):
    print(t)
    t.draw()

#### Exercise 1.3 | Chart Parsing with Context Free Grammar

In [5]:
def init_wfst(token, grammar):
    numtokens = len(tokens)
    wfst = [[None for i in range(numtokens+1)] for j in range(numtokens+1)]
    for i in range(numtokens):
        productions = grammar.productions(rhs=token[i])
        wfst[i][i+1] = productions[0].lhs()
    return wfst

def complete_wfst(wfst, tokens, grammar, trace=False):
    index = dict((p.rhs(), p.lhs()) for p in grammar.productions())
    numtokens = len(tokens)
    for span in range(2, numtokens+1):
        for start in range(numtokens+1-span):
            end = start + span
            for mid in range(start+1, end):
                nt1, nt2 = wfst[start][mid], wfst[mid][end]
                if nt1 and nt2 and (nt1,nt2) in index:
                    wfst[start][end] = index[(nt1,nt2)]
                    if trace:
                        print("[%s] %3s [%s] %3s [%s] %3s [==> [%s] %3s] [%s]" % \
                              (start, nt1, mid, nt2, end, start, index[(nt1,nt2)], end))
    return wfst
def display(wfst, tokens):
    print('\nWFST' + ''.join(("%-4d" % i) for i in range (1, len(wfst))))
    for i in range(len(wfst)-1):
        print("%d " % i, end="")
        for j in range(1, len(wfst)):
            print("%-4s" %(wfst[i][j] or '.'), end=" ")
        print()

Dalam WFST posisi kata direkam dengan mengisi sel dalam matriks segitiga: sumbu vertikal akan menunjukkan posisi awal substring, sedangkan sumbu horizontal akan menunjukkan posisi akhir (sehingga tembakan akan muncul di sel dengan koordinat (1, 2)). Sederhananya, dianggap bahwa setiap kata memiliki kategori leksikal yang unik, dan kami akan menyimpan ini (bukan kata) dalam matriks. Jadi sel (1, 2) akan berisi entri V. 

In [6]:
def display(wfst, tokens):
    print('\nWFST ' + ' '.join(("%-4d" % i) for i in range(1, len(wfst))))
    for i in range(len(wfst)-1):
        print("%d  " % i, end=" ")
        for j in range(1, len(wfst)):
            print("%-4s" % (wfst[i][j] or '.'), end=" ")
        print()

In [7]:
tokens = "I shot an elephant in my pajamas".split()
wfst0 = init_wfst(tokens, groucho_grammar)
display(wfst0, tokens)


WFST 1    2    3    4    5    6    7   
0   NP   .    .    .    .    .    .    
1   .    V    .    .    .    .    .    
2   .    .    Det  .    .    .    .    
3   .    .    .    N    .    .    .    
4   .    .    .    .    P    .    .    
5   .    .    .    .    .    Det  .    
6   .    .    .    .    .    .    N    


Untuk setiap kata dalam teks , kita dapat mencari dalam tata bahasa kita termasuk kategori apa

In [8]:
wfst1 = complete_wfst(wfst0, tokens, groucho_grammar)
display(wfst1, tokens)


WFST 1    2    3    4    5    6    7   
0   NP   .    .    .    .    .    .    
1   .    V    .    VP   .    .    .    
2   .    .    Det  NP   .    .    .    
3   .    .    .    N    .    .    .    
4   .    .    .    .    P    .    .    
5   .    .    .    .    .    Det  NP   
6   .    .    .    .    .    .    N    


Tabel tersebut merepresentasikan bahwa kita  memiliki Det di sel (2,3) untuk kata an, dan N di sel (3,4) untuk kata elephant. Untuk an elephant, yang harus dimasukkan ke dalam sel (2,4), terlebih dahulu adalah mencari produksi dari bentuk A → Det N. Berkonsultasi dengan grammar, di sel (2,4) diketahui bahwa NP dapat di enter